# World Bank Data

## This data was taken from the dataset on [Kaggle](https://www.kaggle.com/worldbank/world-development-indicators)

In [19]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [20]:
def readData(filename):
    with open(filename) as f:
        data = pd.read_csv(f)
        
    return data

In [21]:
all_df = readData('world-development-indicators/indicators.csv')

In [25]:
def getSpecificDf(all_df, indicatorcode):
    specific_indicator = all_df[all_df.IndicatorCode == indicatorcode]
    years = list(range(min(all_df.Year), max(all_df.Year)+1))
    col_names = ['CountryCode', 'CountryName'] + years
    specific_indicator = specific_indicator[['CountryCode', 'CountryName', 'Year', 'Value']]
    for year in years:
        specific_indicator[year] = np.nan
    
    for index, row in specific_indicator.iterrows():
        row_value = deepcopy(row.Value)
        specific_indicator.loc[index, row.Year] = row_value

    specific_indicator = specific_indicator[col_names]
    specific_indicator_formatted = specific_indicator.groupby([specific_indicator.CountryCode, specific_indicator.CountryName]).sum()
    specific_indicator_formatted.reset_index(inplace=True)
    specific_indicator_formatted = specific_indicator_formatted.dropna(axis=1, how='all')
    
    return specific_indicator_formatted

## Life Expectancy

In [54]:
life_expectancy = getSpecificDf(all_df, 'SP.DYN.LE00.IN')

In [56]:
life_expectancy.to_csv('world_bank_data/life_expectancy.csv', index=False)

## Refugees by origin

In [44]:
refugees_by_origin = getSpecificDf(all_df, 'SM.POP.REFG.OR')

In [45]:
refugees_by_origin.to_csv('world_bank_data/refugees_by_origin.csv', index=False)

#### Z - scores

In [46]:
refugees_by_origin_z = deepcopy(refugees_by_origin)
for col in refugees_by_origin.columns[2:]:
    refugees_by_origin_z[col] = (refugees_by_origin[col] - refugees_by_origin[col].mean())/refugees_by_origin[col].std(ddof=0)
    
refugees_by_origin_z.to_csv('world_bank_data/refugees_by_origin_z.csv', index=False)